# 원자간 거리 계산

In [1]:
import math
import timeit

from Bio import PDB

In [2]:
repository = PDB.PDBList()
parser = PDB.PDBParser()
repository.retrieve_pdb_file('1TUP', file_format='pdb', pdir='./data')  # XXX
p53_1tup = parser.get_structure('P 53', './data/pdb1tup.ent')

Structure exists: './data\pdb1tup.ent' 


C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6146.
  warnings.warn(
C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6147.
  warnings.warn(
C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6148.
  warnings.warn(
C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 6149.
  warnings.warn(
C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 6171.
  warnings.warn(
C:\Anaconda3\envs\nbcc_on_campus\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous

### 아연(ZN)과 가장 가까운 원자 확인

In [3]:
# 아연 이온의 위치 정보
zns = []
for atom in p53_1tup.get_atoms():
    if atom.element == 'ZN':
        # print(atom, dir(atom), atom.mass, atom.element, atom.coord[0])
        zns.append(atom)
        
for zn in zns:
        print(zn, zn.coord)

<Atom ZN> [58.108 23.242 57.424]
<Atom ZN> [60.108 17.981 75.931]
<Atom ZN> [33.653  0.403 74.115]


In [4]:
# 거리 계산 함수
def get_closest_atoms(pdb_struct, ref_atom, distance):
    atoms = {}
    rx, ry, rz = ref_atom.coord
    for atom in pdb_struct.get_atoms():
        if atom == ref_atom:
            continue
        x, y, z = atom.coord
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance:
            atoms[atom] = my_dist
    return atoms

In [5]:
# ZN에서 4 A 이내의 원자 검색
for zn in zns:
    print()
    print('ZN', zn.coord)
    atoms = get_closest_atoms(p53_1tup, zn, 4)
    for atom, distance in atoms.items():
        res = atom.parent
        chain = res.parent
        print(chain.id, res.resname, atom.element, distance, atom.coord)


ZN [58.108 23.242 57.424]
A CYS C 3.4080117696286854 [57.77  21.214 60.142]
A CYS S 2.3262243799594877 [57.065 21.452 58.482]
A HIS C 3.4566537492335123 [58.886 20.867 55.036]
A HIS C 3.064120559761192 [58.047 22.038 54.607]
A HIS N 1.9918273537290707 [57.755 23.073 55.471]
A HIS C 2.9243719601324525 [56.993 23.943 54.813]
A CYS C 3.857729198122736 [61.148 25.061 55.897]
A CYS C 3.62725094648044 [61.61  24.087 57.001]
A CYS S 2.2789209624943494 [60.317 23.318 57.979]
A CYS C 3.087214470667822 [57.205 25.099 59.719]
A CYS S 2.2253158446520818 [56.914 25.054 57.917]

ZN [60.108 17.981 75.931]
B CYS C 3.41769274437124 [57.593 15.783 75.207]
B CYS S 2.3254721582053093 [58.586 17.082 74.42 ]
B HIS C 3.4672070967122894 [62.272 17.174 73.345]
B HIS C 3.1139134725185587 [62.061 18.615 73.59 ]
B HIS N 2.0564599972249455 [61.366 19.056 74.71 ]
B HIS C 2.985233217423681 [61.332 20.382 74.647]
B CYS C 3.805126390272999 [62.573 18.263 78.816]
B CYS C 3.1803200512467478 [61.521 17.136 78.652]
B CYS

### 함수 차이로 인한 계산량 변화 확인

In [6]:
nexecs = 10

def get_closest_alternative(pdb_struct, ref_atom, distance):
    atoms = {}
    rx, ry, rz = ref_atom.coord
    for atom in pdb_struct.get_atoms():
        if atom == ref_atom:
            continue
        x, y, z = atom.coord
        if abs(x - rx) > distance or abs(y - ry) > distance or abs(z - rz) > distance:
            continue
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance:
            atoms[atom] = my_dist
    return atoms

In [7]:
# alternative 버전은 계산량을 줄이기 위해 if 조건을 사용하지만
# 거리가 늘어나면 오히려 계산량을 더 늘릴 수 있다.

print('Standard')
for distance in [1, 4, 16, 64, 128]:
    print(timeit.timeit('get_closest_atoms(p53_1tup, zns[0], distance)',
                        'from __main__ import get_closest_atoms, p53_1tup, zns, distance',
                        number=nexecs) / nexecs * 1000)

print()
print('Optimized')
for distance in [1, 4, 16, 64, 128]:
    print(timeit.timeit('get_closest_alternative(p53_1tup, zns[0], distance)',
                        'from __main__ import get_closest_alternative, p53_1tup, zns, distance',
                        number=nexecs) / nexecs * 1000)

Standard
56.86097000000001
56.16990999999997
56.09104
61.81332999999998
61.817810000000016

Optimized
22.163680000000063
24.187129999999968
36.51209999999993
96.81366999999996
95.46046000000005
